## Installation

Warning: current version of `maraboupy` is not supported on Windows


In [1]:
!pip3 install maraboupy

Defaulting to user installation because normal site-packages is not writeable


#### Importing Necessary Packages


In [2]:
import json
import numpy as np
from maraboupy import Marabou, MarabouCore, MarabouUtils
import pandas
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

/home/saimadhavang/.local/lib/python3.10/site-packages/maraboupy/Marabou.py:27: UserWarning: Tensorflow parser is unavailable because tensorflow package is not installed
  warnings.warn("Tensorflow parser is unavailable because tensorflow package is not installed")


## Global Variables

You can set values to the following global variables depending on your desired configuration.

- `NET_PATH`: Path to the model trained on MNIST, in ONNX format
- `MAX_TIME`: Maximum time (in seconds) before a particular instance is timed out
- `START` and `END`: Images corresponding to the indices between `START` and `END` in the MNIST test data will be considered for verification
- `EPSILON`: $\epsilon$ of $\ell_\infty$ norm around each instance


In [10]:
NET_PATH = 'mnist_fc.onnx'
MAX_TIME = 30
START = 0
END = 20
EPSILON = 0.2

M_OPTIONS: MarabouCore.Options = Marabou.createOptions(verbosity=0, timeoutInSeconds=MAX_TIME)

`init_network` initializes the network by loading it and establishing lower and upper bounds for each input neuron depending on the datapoint and the value of $\epsilon$


In [4]:
def init_network(data_point=None, epsilon=None)->Marabou.MarabouNetworkNNet:

    network:Marabou.MarabouNetworkNNet = Marabou.read_onnx(NET_PATH)
    
    for i in range(784):
        network.setLowerBound(i, max(data_point[i]-epsilon, 0.))
        network.setUpperBound(i, min(data_point[i]+epsilon, 1.))
  
    return network

`add_counter_example_constraint` is responsible for adding a constraint on the output that dictates value of `other_label` should be greater than value of `label`.

This way, if we get a `SAT`, we can conclude a counter example has been found; if we get an `UNSAT`, the example has been safely verified


In [5]:
def add_counter_example_constraint(network, label, other_label):

    offset = network.outputVars[0][0][0]

    constraint = MarabouUtils.Equation(MarabouCore.Equation.GE)
    constraint.addAddend(1, other_label+offset)
    constraint.addAddend(-1, label+offset)
    constraint.setScalar(0.0001)

    network.addEquation(constraint)

    return network

Loading and processing data


In [6]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.,), (1.,))
])

trainset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = DataLoader(trainset, batch_size=10000, shuffle=False)

imgs, labels = next(iter(testloader))

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:01<00:00, 5605665.25it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 1175287.37it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 2356924.78it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3562178.15it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



The `main` function, goes through each instance, initializes the network, adds counter example constraints and calls `net.solve()`


In [11]:
def main(output_path):
    res = [[-1.]*10 for i in range(START, END)]

    for i in tqdm(range(START, END)):
        data_point = imgs[i].reshape((784)).numpy()
        label = labels[i].item()

        for other_label in range(10):
            if other_label == int(label):
                continue
            net = init_network(data_point, EPSILON)
            net = add_counter_example_constraint(net, label, other_label)
            exit_code, vals, stats = net.solve(options=M_OPTIONS)
            running_time = stats.getTotalTimeInMicro()
            if exit_code=="sat":
                res[i-START][other_label] = "SAT:{}".format(running_time/10**6)
            elif exit_code=="unsat":
                res[i-START][other_label] = "UNS:{}".format(running_time/10**6)
            else:
                res[i-START][other_label] = exit_code

    res = pandas.DataFrame(res)
    res.to_csv(output_path)

In [12]:
main("./marabou_result.csv")

  0%|          | 0/20 [00:00<?, ?it/s]

sat
input 0 = 0.2
input 1 = 0.2
input 2 = 0.2
input 3 = 0.2
input 4 = 0.2
input 5 = 0.2
input 6 = 0.2
input 7 = 0.0
input 8 = 0.2
input 9 = 0.2
input 10 = 0.2
input 11 = 0.2
input 12 = 0.0
input 13 = 0.2
input 14 = 0.2
input 15 = 0.2
input 16 = 0.2
input 17 = 0.2
input 18 = 0.2
input 19 = 0.2
input 20 = 0.2
input 21 = 0.2
input 22 = 0.2
input 23 = 0.2
input 24 = 0.2
input 25 = 0.2
input 26 = 0.2
input 27 = 0.2
input 28 = 0.2
input 29 = 0.2
input 30 = 0.2
input 31 = 0.2
input 32 = 0.0
input 33 = 0.0
input 34 = 0.0
input 35 = 0.2
input 36 = 0.2
input 37 = 0.2
input 38 = 0.2
input 39 = 0.2
input 40 = 0.2
input 41 = 0.2
input 42 = 0.2
input 43 = 0.2
input 44 = 0.2
input 45 = 0.2
input 46 = 0.2
input 47 = 0.0
input 48 = 0.2
input 49 = 0.2
input 50 = 0.2
input 51 = 0.2
input 52 = 0.2
input 53 = 0.0
input 54 = 0.2
input 55 = 0.2
input 56 = 0.2
input 57 = 0.0
input 58 = 0.2
input 59 = 0.2
input 60 = 0.2
input 61 = 0.2
input 62 = 0.2
input 63 = 0.2
input 64 = 0.2
input 65 = 0.2
input 66 = 0.2
i